# INDEXING AND SELECTING DATA

pandas documentation Indexing 파트의 첫마디.
>The axis labeling information in pandas objects serves many purposes:
- Identifies data (i.e. provides metadata) using known indicators, important for analysis, visualization, and interactive console display. **데이터 구조**를 알려줌
- Enables automatic and explicit data alignment. **피벗테이블** 역할을 함
- Allows intuitive getting and setting of subsets of the data set. **잘라내기** 편함

In this section, we will focus on the final point: namely, how to slice, dice, and generally get and set subsets of pandas objects. The primary focus will be on Series and DataFrame as they have received more development attention in this area.

## 1.Different choices for indexing :`.loc[]`, `.iloc[]`

Getting values from an object with multi-axes selection uses the following notation (using .loc as an example, but the following applies to .iloc as well). Any of the axes accessors may be the **null slice** `:`. Axes left out of the specification are assumed to be :, e.g. p.loc['a'] is equivalent to p.loc['a', :, :].

object type  // indexers

`Series`  //   `s.loc[indexer]`


`DataFrame` // `df.loc[row_indexer, column_indexer]`

### Selection by label: 

`.loc` is primarily label based, but may also be used with a boolean array. .loc will raise KeyError when the items are not found. 

Allowed inputs are:

- single label `.`
- list or array of labels `[]`
- slice object with labels `['a':'f']`
- boolean array, 
- callable function with one argument `[lambda]`



### Selection by Position: 
`.iloc` is primarily integer position based (from 0 to length-1 of the axis), but may also be used with a boolean array.

.iloc will raise IndexError if a requested indexer is out-of-bounds, except slice indexers which allow out-of-bounds indexing. (this conforms with Python/NumPy slice semantics).

Allowed inputs are:

- an integer
- a list or array of integers
- a slice object with ints 1:7
- boolean array
- `callable` function lambda


.loc, .iloc, and also [] indexing can accept a callable as indexer. See more at Selection By Callable.

---
## 2.Basics : `__getitem__` **Lower-dimensional slices**

As mentioned when introducing the data structures in the last section, the primary function of indexing with `[]` (a.k.a. `__getitem__` for those familiar with implementing class behavior in Python) is selecting out **lower-dimensional** slices. The following table shows return type values when indexing pandas objects with []:

Object//Selection//Return Value Type

`Series`//`series[label]`//scalar value

`DataFrame`//`df[col_name]`//`Series` corresponding to colname

In [1]:
import numpy as np
import pandas as pd

In [2]:
dates = pd.date_range('1/1/2020', periods=8)
df = pd.DataFrame(np.random.randn(8, 4), 
                  index=dates, columns=list('ABCD'))


In [3]:
df

,A,B,C,D
2020-01-01,-0.030626,-0.242842,1.296489,0.269898
2020-01-02,0.713899,1.399107,2.329505,-0.969610
2020-01-03,0.812415,0.260762,0.012880,0.656535
2020-01-04,-1.598383,-0.588774,-1.011517,-0.109412
2020-01-05,0.115882,-1.343213,1.647055,-0.317083
2020-01-06,-0.944448,-1.279091,-1.838151,0.567906
2020-01-07,-0.178177,0.283893,-0.417608,-0.407040
2020-01-08,0.404786,1.703493,1.054387,0.878882


In [4]:
s = df['A']

In [5]:
s

2020-01-01   -0.030626
2020-01-02    0.713899
2020-01-03    0.812415
2020-01-04   -1.598383
2020-01-05    0.115882
2020-01-06   -0.944448
2020-01-07   -0.178177
2020-01-08    0.404786
Freq: D, Name: A, dtype: float64

In [6]:
s[dates[5]]

-0.944448086018942

You can pass **a list of columns** to [] to select columns in that order. 

If a column is not contained in the DataFrame, an exception will be raised. 

Multiple columns can also be set in this manner:

In [7]:
df_swap= df.copy()
df_swap[  ['A', 'B']  ] = ['left', 'right']

In [8]:
df_swap

,A,B,C,D
2020-01-01,left,right,1.296489,0.269898
2020-01-02,left,right,2.329505,-0.969610
2020-01-03,left,right,0.012880,0.656535
2020-01-04,left,right,-1.011517,-0.109412
2020-01-05,left,right,1.647055,-0.317083
2020-01-06,left,right,-1.838151,0.567906
2020-01-07,left,right,-0.417608,-0.407040
2020-01-08,left,right,1.054387,0.878882


In [9]:
df_swap[   ['B', 'A']   ] = df_swap[  ['A', 'B']  ]

In [10]:
df_swap

,A,B,C,D
2020-01-01,right,left,1.296489,0.269898
2020-01-02,right,left,2.329505,-0.969610
2020-01-03,right,left,0.012880,0.656535
2020-01-04,right,left,-1.011517,-0.109412
2020-01-05,right,left,1.647055,-0.317083
2020-01-06,right,left,-1.838151,0.567906
2020-01-07,right,left,-0.417608,-0.407040
2020-01-08,right,left,1.054387,0.878882


>**Warning** pandas aligns all AXES when setting Series and DataFrame from .loc, and .iloc.
This will not modify df because the **column alignment is before value assignment.**

In [11]:
df_swap= df.copy()
df_swap[  ['A', 'B']  ] = ['left', 'right']

In [12]:
df_swap[  ['A', 'B']  ]

,A,B
2020-01-01,left,right
2020-01-02,left,right
2020-01-03,left,right
2020-01-04,left,right
2020-01-05,left,right
2020-01-06,left,right
2020-01-07,left,right
2020-01-08,left,right


In [13]:
df_swap.loc[:,  ['B', 'A']]  =  df_swap[  ['A', 'B']]  # column alignment (LHS도 df, RHS도 df.....LHS의 칼럼'A'와, RHS의 칼럼 'A' 맞춘 다음 Set value... )
df_swap[  ['A',  'B']  ]

,A,B
2020-01-01,left,right
2020-01-02,left,right
2020-01-03,left,right
2020-01-04,left,right
2020-01-05,left,right
2020-01-06,left,right
2020-01-07,left,right
2020-01-08,left,right


The correct way to swap column values is by using **raw values**

In [14]:
df_swap.loc[:, ['B', 'A'] ] = df_swap[ ['A','B'] ].to_numpy()  # df vs df로 set하지 말고.... df vs raw vlaue로 set하는 습관!
df_swap[ ['A', 'B']]

,A,B
2020-01-01,right,left
2020-01-02,right,left
2020-01-03,right,left
2020-01-04,right,left
2020-01-05,right,left
2020-01-06,right,left
2020-01-07,right,left
2020-01-08,right,left


---
## 3.Attribute access: `.colname`

---
## 4.Slicing ranges with `[]` operator

The most robust and consistent way of slicing ranges along arbitrary axes is described in the Selection by Position section detailing the .iloc method. For now, we explain the semantics of slicing using the [] operator.

With Series, the syntax works exactly as with an ndarray, returning a slice of the values and the corresponding labels:

In [15]:
s[:5]

2020-01-01   -0.030626
2020-01-02    0.713899
2020-01-03    0.812415
2020-01-04   -1.598383
2020-01-05    0.115882
Freq: D, Name: A, dtype: float64

In [16]:
s[::2]

2020-01-01   -0.030626
2020-01-03    0.812415
2020-01-05    0.115882
2020-01-07   -0.178177
Freq: 2D, Name: A, dtype: float64

In [17]:
s[::-1]

2020-01-08    0.404786
2020-01-07   -0.178177
2020-01-06   -0.944448
2020-01-05    0.115882
2020-01-04   -1.598383
2020-01-03    0.812415
2020-01-02    0.713899
2020-01-01   -0.030626
Freq: -1D, Name: A, dtype: float64

Note that setting works as well:

In [18]:
s2 = s.copy()

In [19]:
s2[:5] = 0
s2

2020-01-01    0.000000
2020-01-02    0.000000
2020-01-03    0.000000
2020-01-04    0.000000
2020-01-05    0.000000
2020-01-06   -0.944448
2020-01-07   -0.178177
2020-01-08    0.404786
Freq: D, Name: A, dtype: float64

With DataFrame, slicing inside of `[]` **slices the rows**. This is provided largely as a convenience since it is such a common operation.

In [20]:
df[:3]

,A,B,C,D
2020-01-01,-0.030626,-0.242842,1.296489,0.269898
2020-01-02,0.713899,1.399107,2.329505,-0.969610
2020-01-03,0.812415,0.260762,0.012880,0.656535


In [21]:
df[::-1]

,A,B,C,D
2020-01-08,0.404786,1.703493,1.054387,0.878882
2020-01-07,-0.178177,0.283893,-0.417608,-0.407040
2020-01-06,-0.944448,-1.279091,-1.838151,0.567906
2020-01-05,0.115882,-1.343213,1.647055,-0.317083
2020-01-04,-1.598383,-0.588774,-1.011517,-0.109412
2020-01-03,0.812415,0.260762,0.012880,0.656535
2020-01-02,0.713899,1.399107,2.329505,-0.969610
2020-01-01,-0.030626,-0.242842,1.296489,0.269898


---
## 5.Selection by label: `.loc`

pandas는 purely label based indexing을 지향한다. 이러한 인덱싱 기법은 엄격한 기준을 따라 작동하게 됨. 
- 인덱싱하는 모든 label은 데이터의 index에 포함되어 있어야 하며, 그렇지 않은 경우 KeyError를 띄운다. 
- 슬라이싱의 경우, 시작점과 종료지점은 모두 included 이다. 
- Integer 형식도 valid labels이지만, 이는 position base가 아님을 유념해야 한다.

.loc 을 통해 엑셀과 유사한 indexing을 할 수 있다. 
 * a single label 'a'
 * a list or array of labels ['a','b','c']
 * a slice object with labels 'a':'f'
 * a boolean array
 * a callable

In [22]:
s1 = pd.Series(np.random.randn(6), index=list('abcdef'))

In [23]:
s1

a    0.635112
b    1.134142
c    0.569837
d   -0.006208
e    0.664319
f   -0.833807
dtype: float64

In [24]:
s1.loc['b']

1.1341418644175691

In [25]:
s1.loc['c':]

c    0.569837
d   -0.006208
e    0.664319
f   -0.833807
dtype: float64

Note that setting works as well:

In [26]:
s1.loc['c':] = 0
s1

a    0.635112
b    1.134142
c    0.000000
d    0.000000
e    0.000000
f    0.000000
dtype: float64

With a DataFrame:

In [27]:
df1 = pd.DataFrame(np.random.randn(6,4),
                  index=list('abcdef'),
                  columns=list('ABCD'))

In [28]:
df1

,A,B,C,D
a,-0.659073,-0.481500,0.637758,0.153752
b,-0.651794,0.444879,1.127866,-0.996273
c,-1.162703,-0.363414,1.016430,1.303238
d,0.511460,-1.392088,2.038586,-0.927960
e,-0.072145,0.593095,2.393348,0.754085
f,-0.741600,0.928139,0.158540,0.682864


In [29]:
df1.loc[['a','c','d']]

,A,B,C,D
a,-0.659073,-0.481500,0.637758,0.153752
c,-1.162703,-0.363414,1.016430,1.303238
d,0.511460,-1.392088,2.038586,-0.927960


In [30]:
df1.loc[['a','b']]

,A,B,C,D
a,-0.659073,-0.481500,0.637758,0.153752
b,-0.651794,0.444879,1.127866,-0.996273


In [31]:
df1.loc[['a','b'], :]  #explicit column is better

,A,B,C,D
a,-0.659073,-0.481500,0.637758,0.153752
b,-0.651794,0.444879,1.127866,-0.996273


Accessing via label slices:

In [32]:
df1.loc['d':, 'A':'C']

,A,B,C
d,0.511460,-1.392088,2.038586
e,-0.072145,0.593095,2.393348
f,-0.741600,0.928139,0.158540


For getting a cross section(reduce to a Series) using a label

In [33]:
df1.loc['a']

A   -0.659073
B   -0.481500
C    0.637758
D    0.153752
Name: a, dtype: float64

For getting values with a boolean array:

In [34]:
df1.loc['a']>0

A    False
B    False
C     True
D     True
Name: a, dtype: bool

In [35]:
df1.loc[:, df1.loc['a'] > 0]

,C,D
a,0.637758,0.153752
b,1.127866,-0.996273
c,1.016430,1.303238
d,2.038586,-0.927960
e,2.393348,0.754085
f,0.158540,0.682864


For getting a value explicitly..(equiv to deprecated df.get_value('a','A')

In [36]:
df1.loc['a','A']

-0.6590730104532766

#### Slicing With Labels: `.loc[row_indexer, col_indexer]`
When using .loc with slices, if both the start and the stop labels are present in the index, then elements _located_ between the two (including them) are returned.

In [37]:
s = pd.Series(list('abcde'), index=[0,3,2,5,4])

In [38]:
s

0    a
3    b
2    c
5    d
4    e
dtype: object

In [39]:
s.loc[3:4]

3    b
2    c
5    d
4    e
dtype: object

If at least one of the two is absent, but the index is **sorted**, and can be compared against start and stop labels, then slicing will still work as expected, by selecting labels which _rank_ between the two:

In [40]:
s.sort_index()

0    a
2    c
3    b
4    e
5    d
dtype: object

In [41]:
s  # s는 그대로 있음

0    a
3    b
2    c
5    d
4    e
dtype: object

In [42]:
s.sort_index().loc[1:6]

2    c
3    b
4    e
5    d
dtype: object

However, if at least one of the two is absent and the index is not sorted, an error will be raised (since doing otherwise would be computationally expensive, as well as potentially ambiguous for mixed type indexes). For instance, in the above example, s.loc[1:6] would raise KeyError.

For the rationale behind this behavior, see [Endpoints are inclusive](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#advanced-endpoints-are-inclusive).

---
## 6.Selection by position

pandas는 Integer-based indexing도 지원함(엑셀의 참조 방법이 두가지인 것처럼)
Numpy나 Python의 slicing은 Position based indexing을 지향함. 단순하긴 한데, 코드 작성자가 데이터구조를 고려해 Axis를 정해줘야 해서 불편함.
직관적인 label-based indexing(if included..)을 애용합시다.
더욱이나 Integer-based indexing은 start bound is included, while the upper bound is excluded.라는 점을 유념해야 함.


In [43]:
s1 = pd.Series(np.random.randn(5), index=list(range(0, 10, 2)))

In [44]:
s1

0    0.432851
2    1.651687
4    0.133432
6   -0.218117
8   -0.934482
dtype: float64

In [45]:
s1.iloc[:3]

0    0.432851
2    1.651687
4    0.133432
dtype: float64

In [46]:
s1.iloc[3]

-0.21811659358195915

Note that setting works as well:

In [47]:
s1.iloc[:3] = 0

In [48]:
s1

0    0.000000
2    0.000000
4    0.000000
6   -0.218117
8   -0.934482
dtype: float64

With a DataFrame:

In [49]:
df1 = pd.DataFrame(np.random.randn(6,4),
                  index=list(range(0, 12, 2)),
                  columns=list(range(0, 8, 2)))


In [50]:
df1

,0,2,4,6
0,-0.290338,0.009397,-0.452871,0.123546
2,-0.113029,2.538586,0.846017,0.514705
4,0.560926,-0.441231,0.006512,-0.879332
6,0.582386,0.525041,-0.303021,0.297694
8,-1.651936,-1.220675,-0.870852,0.727633
10,0.745226,0.532520,0.301498,0.871145


Select via integer slicing:

In [51]:
df1.iloc[:3]

,0,2,4,6
0,-0.290338,0.009397,-0.452871,0.123546
2,-0.113029,2.538586,0.846017,0.514705
4,0.560926,-0.441231,0.006512,-0.879332


In [52]:
df1.iloc[1:5, 2:4]

,4,6
2,0.846017,0.514705
4,0.006512,-0.879332
6,-0.303021,0.297694
8,-0.870852,0.727633


Select via integer list:

In [53]:
df1.iloc[[1,3,5], [1,3]]

,2,6
2,2.538586,0.514705
6,0.525041,0.297694
10,0.532520,0.871145


In [54]:
df1.iloc[1:3, :] #1포지션부터 +2개 행, 모든열

,0,2,4,6
2,-0.113029,2.538586,0.846017,0.514705
4,0.560926,-0.441231,0.006512,-0.879332


In [55]:
df1.iloc[:, 1:3] #모든 행, 1포지션부터 2개열

,2,4
0,0.009397,-0.452871
2,2.538586,0.846017
4,-0.441231,0.006512
6,0.525041,-0.303021
8,-1.220675,-0.870852
10,0.532520,0.301498


In [56]:
df1.iloc[1]  #1포지션 행

0   -0.113029
2    2.538586
4    0.846017
6    0.514705
Name: 2, dtype: float64

Out of range slice indexes are handled gracefully just as in Python/Numpy

In [57]:
x = list('abcdef')

In [58]:
x

['a', 'b', 'c', 'd', 'e', 'f']

In [59]:
x[4:10]  #포지션4('e')부터 길이는 최대 2개이므로 4:6이 엄밀하게 맞는 코드이나, 6개를 불러오라 해도 알아서 끝까지만 탐색

['e', 'f']

In [60]:
x[8:10]

[]

In [61]:
s=pd.Series(x)

In [62]:
s

0    a
1    b
2    c
3    d
4    e
5    f
dtype: object

In [63]:
s.iloc[4:10]

4    e
5    f
dtype: object

In [64]:
s.iloc[8:10]

Series([], dtype: object)

Note that using slices that go out of bounds can result in an empty axis.(an empty Df being returned)

In [65]:
dfl = pd.DataFrame(np.random.randn(5, 2), columns=list('AB'))

In [66]:
dfl

,A,B
0,-1.119634,0.997752
1,-1.219501,0.066257
2,1.112007,-1.782724
3,-0.336499,-0.284180
4,0.383318,-0.760607


In [67]:
dfl.iloc[:, 2:3] #모든 행, 2포지션에서 1개열 뽑아와.... 근데 2포지션? 없는데? 열이 2개니까 1포지션까지만 있지(0,1)

""
0
1
2
3
4


In [68]:
dfl.iloc[:, 1:3]

,B
0,0.997752
1,0.066257
2,-1.782724
3,-0.284180
4,-0.760607


In [69]:
dfl.iloc[4:6] #4포 행에서 2개 행 뽑아와... 근데 행개수 5개인데...

,A,B
4,0.383318,-0.760607


A single indexer that is out of bounds will raise an `IndexError`. A list of indexers where any element is out of bounds will raise an `indexerror`

In [70]:
dfl.iloc[5]

IndexError: single positional indexer is out-of-bounds

In [71]:
dfl.iloc[[4,5,6]]

IndexError: positional indexers are out-of-bounds

In [72]:
dfl.iloc[:, 4]

IndexError: single positional indexer is out-of-bounds

---
## 8. Selection by callable
.loc, .iloc, and also [] indexing can accept a **callable as indexer**. The callable must be a function with one argument(the calling Series or DataFrame) that returns valid output for indexing.

In [73]:
df1 = pd.DataFrame(np.random.randn(6,4),
                  index=list('abcdef'),
                  columns=list('ABCD'))
df1

,A,B,C,D
a,0.431701,-1.024128,-1.043657,0.566217
b,1.232776,-2.008557,0.053515,1.118287
c,0.827704,-1.158416,-0.103864,0.294394
d,-1.004463,-1.023523,-0.389382,-0.593574
e,0.005550,1.026409,1.233130,1.191293
f,1.937430,-0.251924,0.058054,-0.528854


In [74]:
df1.loc[lambda df: df.A > 0, :] #행은 콜러블...콜링 df1 자기자신...A열... 0보다 큰 행...., 모든 열을 불러와...

,A,B,C,D
a,0.431701,-1.024128,-1.043657,0.566217
b,1.232776,-2.008557,0.053515,1.118287
c,0.827704,-1.158416,-0.103864,0.294394
e,0.005550,1.026409,1.233130,1.191293
f,1.937430,-0.251924,0.058054,-0.528854


In [75]:
df1.loc[df1.A>0, :] #equivalent Selection by boolean array

,A,B,C,D
a,0.431701,-1.024128,-1.043657,0.566217
b,1.232776,-2.008557,0.053515,1.118287
c,0.827704,-1.158416,-0.103864,0.294394
e,0.005550,1.026409,1.233130,1.191293
f,1.937430,-0.251924,0.058054,-0.528854


In [76]:
df1.loc[:, lambda df: ['A', 'B']]

,A,B
a,0.431701,-1.024128
b,1.232776,-2.008557
c,0.827704,-1.158416
d,-1.004463,-1.023523
e,0.005550,1.026409
f,1.937430,-0.251924


In [77]:
df1.iloc[:, lambda df: [0,1]]

,A,B
a,0.431701,-1.024128
b,1.232776,-2.008557
c,0.827704,-1.158416
d,-1.004463,-1.023523
e,0.005550,1.026409
f,1.937430,-0.251924


In [78]:
df1[lambda df: df.columns[0]] # callable return... 'A'

a    0.431701
b    1.232776
c    0.827704
d   -1.004463
e    0.005550
f    1.937430
Name: A, dtype: float64

Use callable indexing in Series

In [79]:
df1.A.loc[lambda s: s> 0]

a    0.431701
b    1.232776
c    0.827704
e    0.005550
f    1.937430
Name: A, dtype: float64

Using these methods.. you can chain data selection operations without using a temporary variable...**AMAZING!!!**

In [80]:
bb= pd.read_csv('data/baseball.csv', index_col='id')

In [81]:
(bb.groupby(['year', 'team'])).sum().loc[lambda df: df.r > 100]

stint    g    ab    r    h  X2b  X3b  hr    rbi    sb   cs   bb  \
year team                                                                    
2007 CIN       6  379   745  101  203   35    2  36  125.0  10.0  1.0  105   
     DET       5  301  1062  162  283   54    4  37  144.0  24.0  7.0   97   
     HOU       4  311   926  109  218   47    6  14   77.0  10.0  4.0   60   
     LAN      11  413  1021  153  293   61    3  36  154.0   7.0  5.0  114   
     NYN      13  622  1854  240  509  101    3  61  243.0  22.0  4.0  174   
     SFN       5  482  1305  198  337   67    6  40  171.0  26.0  7.0  235   
     TEX       2  198   729  115  200   40    4  28  115.0  21.0  4.0   73   
     TOR       4  459  1408  187  378   96    2  58  223.0   4.0  2.0  190   

              so   ibb   hbp    sh    sf  gidp  
year team                                       
2007 CIN   127.0  14.0   1.0   1.0  15.0  18.0  
     DET   176.0   3.0  10.0   4.0   8.0  28.0  
     HOU   212.0   3.0   9.0  16.0   6.0  17.0  
     LAN   141.0   8.0   9.0   3.0   8.0  29.0  
     NYN   310.0  24.0  23.0  18.0  15.0  48.0  
     SFN   188.0  51.0   8.0  16.0   6.0  41.0  
     TEX   140.0   4.0   5.0   2.0   8.0  16.0  
     TOR   265.0  16.0  12.0   4.0  16.0  38.0

---
## 9. IX indexer is deprecated

---
## 10. Indexing with list with missing labels is deprecated

Indexing with list with missing labels is deprecated, in favor of `.reindex`

In [82]:
s=pd.Series([1,2,3])

In [83]:
s

0    1
1    2
2    3
dtype: int64

In [84]:
s.loc[[1,2]]

1    2
2    3
dtype: int64

In [85]:
s.loc[[1,2,3]]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


1    2.0
2    3.0
3    NaN
dtype: float64

### Reindexing.. 
the idiomatic way to achieve selecting potentially not-found elements.

See also [reindexing_basic functionality](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html#basics-reindexing)

In [86]:
s.reindex([1,2,3])

1    2.0
2    3.0
3    NaN
dtype: float64

In [87]:
labels=[1,2,3]

Alternatively, selecting only valid keys..... it is guaranteed to preserve the dtype of the selection

In [88]:
s.loc[s.index.intersection(labels)]

1    2
2    3
dtype: int64

In [89]:
s = pd.Series(np.arange(4), index=['a','a','b','c'])

In [90]:
s

a    0
a    1
b    2
c    3
dtype: int32

In [91]:
labels = ['c','d']

In [92]:
s.reindex(labels)

ValueError: cannot reindex from a duplicate axis

Duplicate axis인 경우..... intersection으로 중복된 axis를 풀어주고... .reindex 하면 됨

In [93]:
s.loc[s.index.intersection(labels)].reindex(labels)

c    3.0
d    NaN
dtype: float64

In [94]:
labels = ['a','d']

In [95]:
s.loc[s.index.intersection(labels)]

a    0
a    1
a    0
a    1
dtype: int32

In [96]:
s.loc[s.index.intersection(labels)].reindex(labels)

ValueError: cannot reindex from a duplicate axis

---
## 11. Selecting random samples

A random selection of rows or columns from a Series or DataFrame with the sample() method. The method will sample rows by default, and accepts a specific number of rows/columns to return, or a fraction of rows.

In [97]:
s=pd.Series([0,1,2,3,4,5])

In [98]:
s.sample()

2    2
dtype: int64

In [99]:
s.sample(n=3)

3    3
4    4
0    0
dtype: int64

In [100]:
s.sample(frac=0.5)

2    2
0    0
5    5
dtype: int64

By default, sample will return each row at most once, but one can also sample with replacement using the replace option:

In [101]:
s.sample(n=6, replace=True)

0    0
1    1
5    5
5    5
0    0
4    4
dtype: int64

By default, each row has an equal probability of being selected, but if you want rows to have different probabilities, you can pass the sample function sampling weights as weights. These weights can be a list, a NumPy array, or a Series, but they must be of the same length as the object you are sampling. Missing values will be treated as a weight of zero, and inf values are not allowed. If weights do not sum to 1, they will be re-normalized by dividing all weights by the sum of the weights. For example:


In [102]:
ex_weights = pd.Series([0, 0, 0.2, 0.2, 0.2, 0.4], index=s.index, name="prob")

In [103]:
ex_weights

0    0.0
1    0.0
2    0.2
3    0.2
4    0.2
5    0.4
Name: prob, dtype: float64

In [104]:
s.sample(n=3, weights=ex_weights)

2    2
5    5
3    3
dtype: int64

When applied to a DF, you can use a column of the DF as sampling weights (provided you ar sampling rows and not columns) by simply passing the name of the column as string

In [105]:
df2 = pd.DataFrame({'col1': [9,8,7,6],
                    'weight_column': [0.5, 0.4, 0.1, 0]})

In [106]:
df2.sample(n=3, weights='weight_column')

,col1,weight_column
1,8,0.4
2,7,0.1
0,9,0.5


sample also allows users to sample columns instead of rows using the axis argument.

In [107]:
df2.sample(n=1, axis=1)

,col1
0,9
1,8
2,7
3,6


Finally, one can also set a seed for sample’s random number generator using the random_state argument, which will accept either an integer (as a seed) or a NumPy RandomState object.

In [108]:
df4 = pd.DataFrame({'col1': [1,2,3], 'col2': [2,3,4]})

In [109]:
df4.sample(2, random_state=2)

,col1,col2
2,3,4
1,2,3


In [110]:
df4.sample(2, random_state=2)

,col1,col2
2,3,4
1,2,3


---
## 12. Setting with enlargement

The .loc/[] operations can perform enlargement when setting a non-existent key for that axis.

In the Series case this is effectively an **appending operation.**

In [111]:
se = pd.Series([1,2,3])

In [112]:
se

0    1
1    2
2    3
dtype: int64

In [113]:
se[5]=5.

In [114]:
se

0    1.0
1    2.0
2    3.0
5    5.0
dtype: float64

A DataFrame can be **enlarged** on either **axis** via .loc.

In [115]:
dfi = pd.DataFrame(np.arange(6).reshape(3,2),
                  columns=['A','B'])

In [116]:
dfi.loc[:, 'C'] = dfi.loc[:, 'A']  # enlarge column

In [117]:
dfi

,A,B,C
0,0,1,0
1,2,3,2
2,4,5,4


In [118]:
dfi.loc[3] = 5  # appending row

In [119]:
dfi

,A,B,C
0,0,1,0
1,2,3,2
2,4,5,4
3,5,5,5


---
## 13. Fast scalar value getting and setting

Since indexing with `[]` must handle a lot of cases (single-label access, slicing, boolean indexing, etc.), it has a bit of overhead in order to figure out what you’re asking for. 

If you only want to access a scalar value, the **fastest** way is to use the `.at` and `.iat` methods, which are implemented on all of the data structures.

Similarly to loc, `.at` provides label based **scalar lookups**, while, `.iat` provides integer based lookups analogously to iloc

In [120]:
s = pd.Series(range(6))

In [121]:
s

0    0
1    1
2    2
3    3
4    4
5    5
dtype: int64

In [122]:
s.iat[5]

5

You can also set using these same indexers.

In [123]:
dates = pd.date_range('2000-01-01', periods=8)

In [124]:
df = pd.DataFrame(np.random.randn(8,4),
                 index=dates, 
                 columns=list('ABCD'))

In [125]:
df.at[dates[5], 'A']

0.2586697960473912

In [126]:
df.loc[dates[5], 'A']

0.2586697960473912

In [127]:
df.iat[3,0]

0.5442505932666805

In [128]:
df.at[dates[5], 'E']=7
df.iat[3,0] = 7

In [129]:
df

,A,B,C,D,E
2000-01-01,-1.198369,0.261563,-0.370634,1.027520,NaN
2000-01-02,-0.262224,-0.635727,0.134348,0.278985,NaN
2000-01-03,1.665591,-0.960688,-0.237492,1.338413,NaN
2000-01-04,7.000000,0.531158,-0.249430,-1.391442,NaN
2000-01-05,-0.138239,-0.349732,0.682277,0.617108,NaN
2000-01-06,0.258670,0.438699,0.473015,-0.749075,7.0
2000-01-07,-0.440985,-0.656652,-0.774176,-0.820217,NaN
2000-01-08,-0.494304,1.103674,-2.288570,0.447695,NaN


`at` may **enlarge** the object in-place as above if the indexer is missing.

In [130]:
df.at[dates[-1] + pd.Timedelta('1 day'), 0] = 7

In [131]:
df

,A,B,C,D,E,0
2000-01-01,-1.198369,0.261563,-0.370634,1.027520,NaN,NaN
2000-01-02,-0.262224,-0.635727,0.134348,0.278985,NaN,NaN
2000-01-03,1.665591,-0.960688,-0.237492,1.338413,NaN,NaN
2000-01-04,7.000000,0.531158,-0.249430,-1.391442,NaN,NaN
2000-01-05,-0.138239,-0.349732,0.682277,0.617108,NaN,NaN
2000-01-06,0.258670,0.438699,0.473015,-0.749075,7.0,NaN
2000-01-07,-0.440985,-0.656652,-0.774176,-0.820217,NaN,NaN
2000-01-08,-0.494304,1.103674,-2.288570,0.447695,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN,NaN,7.0


---
## 14. Boolean indexing
Another common operation is the use of boolean vectors to filter the dat. The operators are | for or, & for and , ~ for not.
These __must__ be grouped by using parentheses, since by default Python will evaluate an expression such as df.A >2 & df.B < 3 as df.A > (2& df.B) < 3, while the desired evaluation order is (df.A>2) & (df.B <3)

In [132]:
s = pd.Series(range(-3,4))

In [133]:
range(-3,4)

range(-3, 4)

In [134]:
s

0   -3
1   -2
2   -1
3    0
4    1
5    2
6    3
dtype: int64

In [135]:
s[s>0]

4    1
5    2
6    3
dtype: int64

In [136]:
s[(s<-1)|(s>0.5)]

0   -3
1   -2
4    1
5    2
6    3
dtype: int64

In [137]:
s[~(s<0)]

3    0
4    1
5    2
6    3
dtype: int64

In [138]:
df[df['A'] > 0]

,A,B,C,D,E,0
2000-01-03,1.665591,-0.960688,-0.237492,1.338413,NaN,NaN
2000-01-04,7.000000,0.531158,-0.249430,-1.391442,NaN,NaN
2000-01-06,0.258670,0.438699,0.473015,-0.749075,7.0,NaN


In [139]:
df2 = pd.DataFrame({'a': ['one', 'one', 'two', 'three', 'two', 'one', 'six'],
                    'b': ['x', 'y', 'y', 'x', 'y', 'x', 'x'],
                    'c': np.random.randn(7)})

In [140]:
df2

,a,b,c
0,one,x,1.818377
1,one,y,0.646845
2,two,y,0.035691
3,three,x,0.161822
4,two,y,-0.272958
5,one,x,1.446663
6,six,x,1.043119


List comprehension and the map methosd of series can also be used to produce more complex criteria

In [141]:
criterion = df2['a'].map(lambda x: x.startswith('t'))

In [142]:
criterion

0    False
1    False
2     True
3     True
4     True
5    False
6    False
Name: a, dtype: bool

In [143]:
df2[criterion]

,a,b,c
2,two,y,0.035691
3,three,x,0.161822
4,two,y,-0.272958


In [144]:
#equivalent but slower
df2[[x.startswith('t') for x in df2['a']]]

,a,b,c
2,two,y,0.035691
3,three,x,0.161822
4,two,y,-0.272958


In [145]:
#Multiple criteria
df2[criterion & (df2['b'] == 'x')]

,a,b,c
3,three,x,0.161822


In [146]:
df2.loc[lambda df: df.a.map(lambda x: x.startswith('t'))]

,a,b,c
2,two,y,0.035691
3,three,x,0.161822
4,two,y,-0.272958


In [147]:
df2.loc[criterion & (df2['b'] == 'x'), 'b':'c']

,b,c
3,x,0.161822


---
## 15. Indexing with `.isin()`: Membership Check
Consider the `isin()` method of `Series`, which returns a boolean vector that is true wherever the Series' **elements exist** in the passed list. 

This allows you to select rows where one or more columns have values you want 

In [148]:
s = pd.Series(np.arange(5), index=np.arange(5)[::-1], dtype='int64')

In [149]:
s

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [150]:
s.isin([2, 4, 6])

4    False
3    False
2     True
1    False
0     True
dtype: bool

In [151]:
s[s.isin([2, 4, 6])] #lookup value

2    2
0    4
dtype: int64

The same method is available for `Index` objects and is useful for the cases when you don’t know which of the sought labels are in fact present:

In [152]:
s[s.index.isin([2, 4, 6])] #lookup index... 근데 존재하는지 알지는 못할 경우

4    0
2    2
dtype: int64

Compare it to the following: Index sort, Non-existing label, dtype

In [153]:
s.reindex([2, 4, 6])

2    2.0
4    0.0
6    NaN
dtype: float64

In addition to that, MultiIndex allows selecting a separate level to use in the membership check:


In [154]:
s_mi = pd.Series(np.arange(6),
                index=pd.MultiIndex.from_product([[0,1],['a','b','c']]))

In [155]:
s_mi

0  a    0
   b    1
   c    2
1  a    3
   b    4
   c    5
dtype: int32

In [156]:
s_mi.iloc[s_mi.index.isin([(1,'a'), (2,'b'), (0,'c')])]

0  c    2
1  a    3
dtype: int32

In [157]:
s_mi.iloc[s_mi.index.isin(['a','c','e'], level=1)]

0  a    0
   c    2
1  a    3
   c    5
dtype: int32

DataFrame also has an isin() method. When calling isin, pass a set of values as either an `array []` OR `dict {}`. If values is an array, isin returns a DataFrame of booleans that is the same shape as the original DataFrame, with True wherever the element is in the sequence of values.

In [158]:
df = pd.DataFrame({'vals': [1, 2, 3, 4], 'ids': ['a', 'b', 'f', 'n'],
                   'ids2': ['a', 'n', 'c', 'n']})

In [159]:
df

,vals,ids,ids2
0,1,a,a
1,2,b,n
2,3,f,c
3,4,n,n


In [160]:
values=['a','b',1,3]

In [161]:
df.isin(values)

,vals,ids,ids2
0,True,True,True
1,False,True,False
2,True,False,False
3,False,False,False


Oftentimes you’ll want to match certain values with certain columns. Just make values a `dict {key: value}` where the key is the column, and the value is a list of items you want to check for.

In [162]:
values={'ids':['a','b'], 'vals':[1,3]}

In [163]:
df.isin(values)

,vals,ids,ids2
0,True,True,False
1,False,True,False
2,True,False,False
3,False,False,False


Combine DataFrame’s isin with the `any()` and `all()` methods to quickly select **subsets** of your data that meet a given criteria. To select a row where each column meets its own criterion:

In [164]:
values = {'ids': ['a', 'b'], 'ids2': ['a', 'c'], 'vals': [1, 3]}

In [165]:
row_mask = df.isin(values).all(1)

In [166]:
row_mask

0     True
1    False
2    False
3    False
dtype: bool

In [167]:
df.loc[row_mask]

,vals,ids,ids2
0,1,a,a


---
## 16. The `.where()` Method and Masking
Selecting values from a series with a boolean vector generally returns a subset of the data. To guarantee that selection output has the __same shape as the original data__(just as what `isin()` returns), you can use the `where()` method in Series and DataFrame.

In [168]:
s

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [169]:
s[s>0]

3    1
2    2
1    3
0    4
dtype: int64

In [170]:
s.where(s>0)

4    NaN
3    1.0
2    2.0
1    3.0
0    4.0
dtype: float64

In [171]:
s.where(s.isin([1,2,3,4]))

4    NaN
3    1.0
2    2.0
1    3.0
0    4.0
dtype: float64

Selecting values from a DF with a boolean criterion now also preserves input data shape. `where()` is used under the hood as the implementation. the code below is equivalent to df.where(df < 0)

In [172]:
dates = pd.date_range('20010101', periods=8)
df = pd.DataFrame(np.random.randn(8,3),
                 index=dates)

In [173]:
df

,0,1,2
2001-01-01,0.636149,1.499046,1.033959
2001-01-02,1.799045,1.959878,-0.325004
2001-01-03,-0.094168,-0.405248,0.768575
2001-01-04,2.639533,-1.212930,-0.416527
2001-01-05,1.152167,0.117167,0.787698
2001-01-06,1.184974,-1.883099,-2.349857
2001-01-07,-0.488627,0.236735,-2.382830
2001-01-08,-0.301514,1.178797,-0.039338


In [174]:
df[df< 0]

,0,1,2
2001-01-01,NaN,NaN,NaN
2001-01-02,NaN,NaN,-0.325004
2001-01-03,-0.094168,-0.405248,NaN
2001-01-04,NaN,-1.212930,-0.416527
2001-01-05,NaN,NaN,NaN
2001-01-06,NaN,-1.883099,-2.349857
2001-01-07,-0.488627,NaN,-2.382830
2001-01-08,-0.301514,NaN,-0.039338


In [175]:
df.where(df<0)

,0,1,2
2001-01-01,NaN,NaN,NaN
2001-01-02,NaN,NaN,-0.325004
2001-01-03,-0.094168,-0.405248,NaN
2001-01-04,NaN,-1.212930,-0.416527
2001-01-05,NaN,NaN,NaN
2001-01-06,NaN,-1.883099,-2.349857
2001-01-07,-0.488627,NaN,-2.382830
2001-01-08,-0.301514,NaN,-0.039338


Compared to `[df<0]`, where() can do something else...

In addition, where takes an optional `other` argument for replacement of values where the condition is False, in the returned copy.

In [176]:
df.where(df<0, other=-df)

,0,1,2
2001-01-01,-0.636149,-1.499046,-1.033959
2001-01-02,-1.799045,-1.959878,-0.325004
2001-01-03,-0.094168,-0.405248,-0.768575
2001-01-04,-2.639533,-1.212930,-0.416527
2001-01-05,-1.152167,-0.117167,-0.787698
2001-01-06,-1.184974,-1.883099,-2.349857
2001-01-07,-0.488627,-0.236735,-2.382830
2001-01-08,-0.301514,-1.178797,-0.039338


You may wish to set values based on some boolean criteria. This can be done intuitively like so:

In [177]:
s2=s.copy()
s2

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [178]:
s2[s2<1] = 999

In [179]:
s2

4    999
3      1
2      2
1      3
0      4
dtype: int64

In [180]:
df2 = df.copy()

In [181]:
df2[df2 <0] = 0

In [182]:
df2

,0,1,2
2001-01-01,0.636149,1.499046,1.033959
2001-01-02,1.799045,1.959878,0.000000
2001-01-03,0.000000,0.000000,0.768575
2001-01-04,2.639533,0.000000,0.000000
2001-01-05,1.152167,0.117167,0.787698
2001-01-06,1.184974,0.000000,0.000000
2001-01-07,0.000000,0.236735,0.000000
2001-01-08,0.000000,1.178797,0.000000


By default, where returns a modified copy of the data. There is an optional parameter inplace so that the original data can be modified without creating a copy:

In [183]:
df_orig = df.copy()

In [184]:
df_orig

,0,1,2
2001-01-01,0.636149,1.499046,1.033959
2001-01-02,1.799045,1.959878,-0.325004
2001-01-03,-0.094168,-0.405248,0.768575
2001-01-04,2.639533,-1.212930,-0.416527
2001-01-05,1.152167,0.117167,0.787698
2001-01-06,1.184974,-1.883099,-2.349857
2001-01-07,-0.488627,0.236735,-2.382830
2001-01-08,-0.301514,1.178797,-0.039338


In [185]:
df_orig.where(df > 0, other=-df, inplace=True)

In [186]:
df_orig

,0,1,2
2001-01-01,0.636149,1.499046,1.033959
2001-01-02,1.799045,1.959878,0.325004
2001-01-03,0.094168,0.405248,0.768575
2001-01-04,2.639533,1.212930,0.416527
2001-01-05,1.152167,0.117167,0.787698
2001-01-06,1.184974,1.883099,2.349857
2001-01-07,0.488627,0.236735,2.382830
2001-01-08,0.301514,1.178797,0.039338


> **NOTE** The signature for DataFrame.where() differs from numpy.where(). Roughly df1.where(m, df2) is equivalent to np.where(m, df1, df2).

In [187]:
df.where(df < 0, -df) == np.where(df < 0, df, -df)

,0,1,2
2001-01-01,True,True,True
2001-01-02,True,True,True
2001-01-03,True,True,True
2001-01-04,True,True,True
2001-01-05,True,True,True
2001-01-06,True,True,True
2001-01-07,True,True,True
2001-01-08,True,True,True


### Alignment
Furthermorem `where` aligns the input boolean condition (ndarray or DataFrame), such that **partial selection** with setting is possible.
이건 판다스의 Label 지향성 때문에 가능한 기능!

In [188]:
df2 = df.copy()

In [189]:
df2

,0,1,2
2001-01-01,0.636149,1.499046,1.033959
2001-01-02,1.799045,1.959878,-0.325004
2001-01-03,-0.094168,-0.405248,0.768575
2001-01-04,2.639533,-1.212930,-0.416527
2001-01-05,1.152167,0.117167,0.787698
2001-01-06,1.184974,-1.883099,-2.349857
2001-01-07,-0.488627,0.236735,-2.382830
2001-01-08,-0.301514,1.178797,-0.039338


In [190]:
df2[1:4]  > 0 

,0,1,2
2001-01-02,True,True,False
2001-01-03,False,False,True
2001-01-04,True,False,False


In [191]:
df2[df2[1:4] > 0] #Align으로 지자리 찾아 박힘

,0,1,2
2001-01-01,NaN,NaN,NaN
2001-01-02,1.799045,1.959878,NaN
2001-01-03,NaN,NaN,0.768575
2001-01-04,2.639533,NaN,NaN
2001-01-05,NaN,NaN,NaN
2001-01-06,NaN,NaN,NaN
2001-01-07,NaN,NaN,NaN
2001-01-08,NaN,NaN,NaN


In [192]:
df2[df2[1:4] > 0] = 'Fuck'  

In [193]:
df2

,0,1,2
2001-01-01,0.636149,1.49905,1.03396
2001-01-02,Fuck,Fuck,-0.325004
2001-01-03,-0.0941678,-0.405248,Fuck
2001-01-04,Fuck,-1.21293,-0.416527
2001-01-05,1.15217,0.117167,0.787698
2001-01-06,1.18497,-1.8831,-2.34986
2001-01-07,-0.488627,0.236735,-2.38283
2001-01-08,-0.301514,1.1788,-0.0393378


`where` can also accept axis and level parameters to align the input when performing the where

In [198]:
df.columns = ['A','B','C']

In [199]:
df2 = df.copy()

In [200]:
cond = df2>0

In [201]:
df2.where(cond)

,A,B,C
2001-01-01,0.636149,1.499046,1.033959
2001-01-02,1.799045,1.959878,NaN
2001-01-03,NaN,NaN,0.768575
2001-01-04,2.639533,NaN,NaN
2001-01-05,1.152167,0.117167,0.787698
2001-01-06,1.184974,NaN,NaN
2001-01-07,NaN,0.236735,NaN
2001-01-08,NaN,1.178797,NaN


In [195]:
df2.where(cond, other=df2['A'], axis = 'index') #other = df2['A']인데 index방향으로 끌어다 쓸지, column 방향으로 끌어다 쓸지 알려줘야지..!

,A,B,C
2001-01-01,-2.540347,-2.540347,-2.540347
2001-01-02,-0.128392,1.047398,-0.128392
2001-01-03,-0.440818,-0.440818,1.157269
2001-01-04,-1.016749,1.398112,-1.016749
2001-01-05,-0.770575,1.294242,-0.770575
2001-01-06,0.670972,0.670972,0.196063
2001-01-07,1.145125,0.938324,0.878154
2001-01-08,-0.251020,0.853689,0.493412


This is equivalent to (but faster than) the following

In [196]:
df.apply(lambda x, y: x.where(x > 0, y), y=df['A'])

,A,B,C
2001-01-01,-2.540347,-2.540347,-2.540347
2001-01-02,-0.128392,1.047398,-0.128392
2001-01-03,-0.440818,-0.440818,1.157269
2001-01-04,-1.016749,1.398112,-1.016749
2001-01-05,-0.770575,1.294242,-0.770575
2001-01-06,0.670972,0.670972,0.196063
2001-01-07,1.145125,0.938324,0.878154
2001-01-08,-0.251020,0.853689,0.493412


### Mask
`mask` is the inverse boolean operation of where

In [197]:
s

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [199]:
s.mask(s >= 3)

4    0.0
3    1.0
2    2.0
1    NaN
0    NaN
dtype: float64

In [200]:
s.where(s >= 3)

4    NaN
3    NaN
2    NaN
1    3.0
0    4.0
dtype: float64

In [201]:
df.mask(df >= 0)

,A,B,C
2001-01-01,-2.540347,-0.007532,-1.194449
2001-01-02,-0.128392,NaN,-0.157081
2001-01-03,-0.440818,-0.697807,NaN
2001-01-04,-1.016749,NaN,-0.336502
2001-01-05,-0.770575,NaN,-0.205928
2001-01-06,NaN,-0.176181,NaN
2001-01-07,NaN,NaN,NaN
2001-01-08,-0.251020,NaN,NaN


---
## 17. The `.query()` Method

---
## 18. Duplicate data

---
## 19. Dictionary-like `.get()` method

---
## 20. the `.lookup()` method

---
## 21. Index object

---
## 22. Set/reset Index

---
## 23. Returning a view versus a copy

When setting values in a pandas object, care must be taken to avoid what is called `chained indexing`. Here is an example

In [203]:
dfmi = pd.DataFrame([list('abcd'),
                     list('efgh'),
                     list('ijkl'),
                     list('mnop')],
                   columns=pd.MultiIndex.from_product([['one','two'],
                                                       ['first','second']]))

In [204]:
dfmi

one          two       
  first second first second
0     a      b     c      d
1     e      f     g      h
2     i      j     k      l
3     m      n     o      p

Compare these two access methods:

In [206]:
dfmi['one']['second']

0    b
1    f
2    j
3    n
Name: second, dtype: object

In [207]:
dfmi.loc[:, ('one', 'second')]

0    b
1    f
2    j
3    n
Name: (one, second), dtype: object

These both yield the same results, so which should you use? It is instructive to understand the order of operations on these and why method 2 (.loc) is much preferred over method 1 (chained []).

`dfmi['one']` selects the first level of the columns and returns a DataFrame that is singly-indexed. Then another Python operation dfmi_with_one['second'] selects the series indexed by 'second'. This is indicated by the variable dfmi_with_one because pandas sees these operations as separate events. e.g. separate calls to __getitem__, so it has to treat them as linear operations, they happen one after another.

Contrast this to df.loc[:,('one','second')] which passes a nested tuple of (slice(None),('one','second')) to a single call to __getitem__. This allows pandas to deal with this as a single entity. Furthermore this order of operations can be significantly faster, and allows one to index both axes if so desired.

In [ ]:
dfmi.loc[:, ('one', 'second')] = 